In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


In [20]:
# Load the data
data = pd.read_csv('bot_detection_data.csv')  # Update this to your file path

# Display the first few rows
data.head()


,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,False,1,Adkinston,2020-05-11 15:29:50,NaN
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,True,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,True,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,True,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,False,1,Camachoville,2020-04-13 21:24:21,foreign mention


In [21]:
imputer = SimpleImputer(strategy='mean')


In [22]:
import re
from sklearn.preprocessing import LabelEncoder

# Text preprocessing function
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    return text

# Apply preprocessing to the tweet column
data['Tweet'] = data['Tweet'].apply(preprocess_text)

# Encode 'Verified' column (assuming it's binary True/False)
data['Verified'] = data['Verified'].apply(lambda x: 1 if x == 'TRUE' else 0)

# Prepare labels and features
X_text = data['Tweet']
y = data['Bot Label']

In [23]:
tfidf = TfidfVectorizer(max_features=1000)
X_text_tfidf = tfidf.fit_transform(X_text)

# Convert sparse matrix to DataFrame for merging with other features
X_text_tfidf_df = pd.DataFrame(X_text_tfidf.toarray(), columns=tfidf.get_feature_names_out())


In [24]:
# Select other relevant features
X_other = data[['Retweet Count', 'Mention Count', 'Follower Count', 'Verified']]
X = pd.concat([X_text_tfidf_df, X_other.reset_index(drop=True)], axis=1)


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [26]:
model = LogisticRegression()
model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [27]:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]  # Probability of being a bot

# Print the classification report
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")


              precision    recall  f1-score   support

           0       0.50      0.51      0.50      4968
           1       0.51      0.50      0.50      5032

    accuracy                           0.50     10000
   macro avg       0.50      0.50      0.50     10000
weighted avg       0.50      0.50      0.50     10000

Accuracy: 0.50


In [28]:
def predict_bot(text, retweet_count, mention_count, follower_count, verified):
    # Preprocess text
    text = preprocess_text(text)
    text_tfidf = tfidf.transform([text])
    text_tfidf_df = pd.DataFrame(text_tfidf.toarray(), columns=tfidf.get_feature_names_out())

    # Create a DataFrame with additional features
    input_features = pd.DataFrame({
        'Retweet Count': [retweet_count],
        'Mention Count': [mention_count],
        'Follower Count': [follower_count],
        'Verified': [verified]
    })

    # Combine text features and additional features
    input_data = pd.concat([text_tfidf_df, input_features], axis=1)

    # Predict probability of being a bot
    bot_prob = model.predict_proba(input_data)[:, 1]
    return bot_prob[0]

# Example usage
text = "This is a sample tweet for bot detection."
print("Bot Probability:", predict_bot(text, 10, 2, 300, 1))


Bot Probability: 0.46680266461202413


In [29]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.pooler_output.detach().numpy()

# Example usage
example_text = "This is a sample tweet for bot detection."
embedding = get_bert_embeddings(example_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [30]:
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512, clean_up_tokenization_spaces=False)
outputs = model(**inputs)


Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.


In [ ]:
# SWAP TO T4 GPU RUNTIME BEFORE RUNNING!!! WILL TAKE HOURS IF NOT
data['embeddings'] = data['Tweet'].apply(get_bert_embeddings)


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Reduce embeddings to 2 dimensions
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(X_text)  # X_text being your BERT embeddings

# Plot with color coding by label
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=y)  # y = bot labels
plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.title("PCA of BERT Embeddings")
plt.show()


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(X_text)  # Compute similarity between embeddings
# Example: Check similarity between the first few samples
print(similarity_matrix[:5, :5])


In [ ]:
# Extract embeddings and other features
X_text = np.vstack(data['embeddings'].values)  # Converts list of embeddings into a 2D array
X_other = data[['Retweet Count', 'Mention Count', 'Follower Count', 'Verified']].values
X = np.hstack((X_text, X_other))  # Combine embeddings with additional features
y = data['Bot Label'].values      # Target labels


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Predictions and evaluation
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")


In [ ]:
# ipython-input-14-efaf1001156a
!pip install transformers
from transformers import AutoTokenizer, AutoModel
import torch

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'  # Or any other BERT variant
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModel.from_pretrained(model_name) # Renamed from 'model' to 'bert_model'

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = bert_model(**inputs) # Use 'bert_model' instead of 'model'
    return outputs.pooler_output.detach().numpy()